In [1]:
# %% includes
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import csv
import datetime as dt
from datetime import timedelta
from datetime import datetime
#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
from pandas import concat
# Import Libraries and packages from Keras
from scipy import stats
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import figure
from scipy.stats.stats import pearsonr


import time
import pylab as pl
from IPython import display
from tabulate import tabulate


C:\Users\ss97\AppData\Local\Temp\ipykernel_22984\2587958859.py:20: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


# r² : BP-corr. coeff

$$ r^2 =  \frac{\sum_{i=1}^n (O_i- \overline{O})(P_i- \overline{P})} {\sqrt{\sum_{i=1}^n(O_i- \overline{O})^2}\sqrt{\sum_{i=1}^n(P_i- \overline{P})^2}}  $$

In [2]:
def BravaisPearson_corr(observed, predicted):
    #means
    mean_observed = observed.mean()
    mean_predicted = predicted.mean()

    #diff between objeckt_i and mean of the objects
    o_diff_part = (observed-mean_observed)
    p_diff_part = (predicted-mean_predicted)

    #counter
    counter_o_p_mult = o_diff_part*p_diff_part
    counter = np.sum(counter_o_p_mult)

    #denominator
    denominator_o_part = np.sqrt(np.sum(np.square(o_diff_part)))
    denominator_p_part = np.sqrt(np.sum(np.square(p_diff_part)))
    denominator = denominator_o_part*denominator_p_part
    r_square = np.sqrt((counter/denominator)*(counter/denominator))
    
    return r_square

# E: Nash Sucliffe efficiency 
## $$ E =  1 - \frac{\sum_{i=1}^n (O_i- P_i)^2} {\sum_{i=1}^n (O_i- \overline{O})^2}  $$
#### $$E \in [1: -\infty )$$ $$ 1: total fit$$  $$-\infty : no fit$$ $$ E < 0 : "worse\_than\_average"$$

In [3]:
def NSE(observed, predicted):

    mean_observed = observed.mean()

    #counter
    counter = np.sum((np.square(observed-predicted)))

    #denominator
    denominator = np.sum(np.square(observed-mean_observed))

    #E

    E = 1-counter/denominator

    return E

# d: Index of agreement

## $$ d =1-  \frac{\sum_{i=1}^n (O_i- P_i)^2} {\sum_{i=1}^n (|P_i- \overline{O}|+|P_i- \overline{O}|)^2}  $$

### $$d \in [1: 0 ]$$ $$ 1: total fit$$  $$0: no fit$$ 


In [4]:
def IoD(observed, predicted):

    mean_observed = observed.mean()

    #counter
    counter = np.sum(np.square(observed-predicted))

    #denominator
    denominator = np.sum(np.square(abs(predicted-mean_observed) + abs(observed-mean_observed)))

    #d
    d = 1 - counter/denominator
    
    return d


# modified E
## $$ E_{mod} =  1 - \frac{\sum_{i=1}^n |O_i- P_i|^J} {\sum_{i=1}^n |O_i- \overline{O}|^J}  $$


In [5]:
def NSE_mod(observed, predicted, J):


    mean_observed = observed.mean()

    #counter
    counter = np.sum(pow(abs(observed-predicted), J))

    #denominator
    denominator = np.sum(pow(abs(observed-mean_observed), J))

    #E

    E = 1-counter/denominator

    return E

# modified d:

## $$ d_{mod} = 1- \frac{\sum_{i=1}^n |O_i- P_i|^J} {\sum_{i=1}^n (|P_i- \overline{O}|+|P_i- \overline{O}|)^J}  $$


In [6]:
def IoD_mod(observed, predicted, J):

    mean_observed = observed.mean()

    #counter
    counter = np.sum(pow(abs(observed-predicted), J))

    #denominator
    denominator = np.sum(pow((abs(predicted-mean_observed) + abs(observed-mean_observed)), J))



    #d
    d = 1 - counter/denominator

    return d


# $$E_{rel}$$
## $$ E_{rel} =  1 - \frac{\sum_{i=1}^n (\frac{ O_i- P_i}{O_i})^2} {\sum_{i=1}^n (\frac{ O_i- \overline{O}}{\overline{O}})^2}  $$


In [7]:
def NSE_rel(observed, predicted):

    mean_observed = observed.mean()

    # counter
    counter = np.sum(pow( (observed-predicted)  / observed ,2))

    #denominator
    denominator = np.sum(pow( (observed-mean_observed)  / mean_observed ,2))

    E_rel = 1 - (counter/denominator)

    return E_rel

# $$d_{rel}$$

## $$ d_{rel} =1-  \frac{\sum_{i=1}^n \frac{(O_i- P_i)^2}{O_i}} {\sum_{i=1}^n (\frac{|P_i- \overline{O}|+|P_i- \overline{O}|}{\overline{O}})^2}  $$


In [8]:
def  IoD_rel(observed, predicted):
    mean_observed = observed.mean()


    #counter
    counter = np.sum(pow((observed-predicted)/observed,2))
    #denominator
    denominator = np.sum(pow(((abs(predicted-mean_observed)+abs(observed-mean_observed))/mean_observed),2))
    #d_rel
    d_rel =1- counter / denominator
    return d_rel

### calling all functions

In [9]:
def execute_evaluation(observed, predicted, J):
    BPC_loc = pearsonr(observed, predicted)[0] # can also become negative since used from scipy
    NSE_loc = NSE(observed, predicted)
    IoD_loc = IoD(observed, predicted)
    NSE_mod_loc = NSE_mod(observed, predicted, J)
    IoD_mod_loc = IoD_mod(observed, predicted, J)
    NSE_rel_loc = NSE_rel(observed, predicted)
    IoD_rel_loc = IoD_rel(observed, predicted)
    return BPC_loc, NSE_loc,IoD_loc, NSE_mod_loc, IoD_mod_loc, NSE_rel_loc, IoD_rel_loc

def name_values(vall_array):
    vall_list = list([
                     ["BravaisPearson_corr",vall_array[0]],
                     ["NSE",vall_array[1]],
                     ["IoD",vall_array[2]],
                     ["NSE_mod",vall_array[3]],
                     ["IoD_mod",vall_array[4]],
                     ["NSE_rel",vall_array[5]],
                     ["IoD_rel",vall_array[6]]
    ])
    return vall_list
    

In [10]:
def read_csv(path):
    return pd.read_csv(path, delimiter=",", engine="python")

In [11]:
def overallMetricsFramework(foldername):
    hours = [2, 3, 4, 8, 12]
    J_mod = 3
    Mesurement_df = pd.DataFrame(index = ["BravaisPearson_corr", "(E)NSE", "(d)IoA", f"(E)NSE_mod_J={J_mod}", f"(d)IoA_mod_J={J_mod}", "(E)NSE_rel", "(d)IoA_rel"], columns =[])
    
    for h in hours:
        df_h_origin = read_csv(f"../final results/{foldername}/Qualitative/{h}hrs/forecasts_{h}h.csv")
        df_h = (df_h_origin.copy()).dropna()
        comparison_df = execute_evaluation(observed = df_h["measured"] , predicted = df_h["predicted"], J = J_mod)  
        Mesurement_df[f'{h}h prediction Sennheutte'] = comparison_df
    
    print(f"\n~~~~~~~~~~~~ {i} ~~~~~~~~~~~~\n")
    print(tabulate(Mesurement_df , headers="keys", tablefmt="psql"))
    return Mesurement_df

# Results

In [12]:
folder = ['STRPM', 'Baseline', 'STRPMr']
for i in folder:
    framework_result = overallMetricsFramework(i)
    framework_result.to_csv(f"../final results/{i}/Quantitative/overall_metrics_{i}.csv", sep =",")


~~~~~~~~~~~~ STRPM ~~~~~~~~~~~~

+---------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------------+
|                     |   2h prediction Sennheutte |   3h prediction Sennheutte |   4h prediction Sennheutte |   8h prediction Sennheutte |   12h prediction Sennheutte |
|---------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------------|
| BravaisPearson_corr |                  0.170208  |                  0.217053  |                  0.151822  |                  0.13388   |                   0.19093   |
| (E)NSE              |                 -0.0362282 |                  0.0233174 |                 -0.0726325 |                 -0.0499517 |                  -0.0407461 |
| (d)IoA              |                  0.322205  |                  0.319001  |                  0.332424  |      